In [14]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from chromadb.config import Settings
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

import os

# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

In [15]:
# specify the directory where Chroma will build the vector db
CHROMA_DB_DIRECTORY='db'

# specify where the source documents are
DOCUMENT_SOURCE_DIRECTORY='/path/to/source/documents'

# settings include things like which database backend chroma will use
# here we will be using duck db
# and document will be stored under db
# with no telemetry (i.e. nothing will be tracked)

CHROMA_SETTINGS = Settings(
    chroma_db_impl='duckdb+parquet',
    persist_directory=CHROMA_DB_DIRECTORY,
    anonymized_telemetry=False
)

# target number of relevant chunks to return 
TARGET_SOURCE_CHUNKS=4

# the number of characters that will make up a chunk
CHUNK_SIZE=3000

# the number of overlapping characters to maintain the chunk
# continuity
CHUNK_OVERLAP=50

# whether to show or hide specific documents while LLM giving response
# i.e. whether we need to show the document sources that our LLM
# referred while giving the answer 

HIDE_SOURCE_DOCUMENTS=False

In [2]:
# Loader
loader = PyPDFLoader("/Users/alexeyvaganov/Downloads/ЗНР_1_+6.pdf")
documents = loader.load()





Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)


In [7]:
documents

[Document(page_content='ЗНР на Требование 9.101 Основной процесс Требование Номер ФТ Отражение хозяйственных операций Формирование отчета. Project Commitment line items - PO WBS. Отчет для отслеживания заказов по WBS 9,101  1. Необходимо скопировать (сделать отдельный отчет) Отчет Overdue Open Commitment в Подсистему Внеоборотные активы – Отчеты по Внеоборотным активам 2. В Отчет необходимо внести изменение и поменять перечень подразделений на Подразделения WBS AUC (сейчас там WBS MI) 3. Необходимо сделать отбор по аналитике статья расходов: «на стоимость внеоборотного атива» (сейчас в отчете другие отборы по статьям – их надо убрать)  \n \n 4. Добавить в колонки отчета дополнительные столбцы:  5. Создать роль на редактирование и просмотр отчета   \n', metadata={'source': '/Users/alexeyvaganov/Downloads/ЗНР_1_+6.pdf', 'page': 0})]

In [4]:
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
# )
# documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 1


In [11]:
def is_pdf(file_path):
    """Проверяет, является ли файл формата PDF."""
    _, extension = os.path.splitext(file_path)
    return extension == '.pdf'

In [13]:
directory_path = '/Users/alexeyvaganov/doc/1C/ZNR'
files = os.listdir(directory_path)

for file in files:
    if os.path.isfile(os.path.join(directory_path, file)):
        if is_pdf(file):
            print("Файл является PDF.")
        else:
            print("Файл не является PDF.")

Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.
Файл является PDF.


In [5]:
#  Загрузка в баз
db = Chroma.from_documents(
    documents,
    hf_embeddings_model,
    client_settings=Settings(anonymized_telemetry=False),
)